### Chen, Chap 2

In [204]:
using SymPy

In [205]:
q = symbols("q", real=true)
m = symbols("m", positive=true)
omega_c = symbols("omega_c", positive=true)

@syms x y z t
@syms bx by bz
@syms ex ey ez

@symfuns vx vy vz

v = [vx, vy, vz]
b = [bx, by, bz]
e = [ex, ey, ez]

3-element Array{SymPy.Sym,1}:
 ex
 ey
 ez

In [206]:
function gyromotion(cur_v=v, cur_b=b, cur_e=e, cur_q=q, cur_m=m)
    
    cur_eq = cross(map(xx->xx(t), cur_v), cur_b)  
    
    cur_eq += cur_e 
    
    cur_eq *= ( cur_q / cur_m )
    
    cur_eq -= map(cur_val -> diff(cur_val(t), t), cur_v)
    
    cur_eq
end

gyromotion (generic function with 6 methods)

In [207]:
gyromotion()

3-element Array{SymPy.Sym,1}:
 -Derivative(vx(t), t) + q*(-by*vz(t) + bz*vy(t) + ex)/m
  -Derivative(vy(t), t) + q*(bx*vz(t) - bz*vx(t) + ey)/m
 -Derivative(vz(t), t) + q*(-bx*vy(t) + by*vx(t) + ez)/m

#### Uniform B w/ E = 0

In [208]:
simple_eq = subs.(gyromotion(), bx=>0, by=>0, ex=>0, ey=>0, ez=>0, q*bz/m => omega_c)

3-element Array{SymPy.Sym,1}:
  omega_c*vy(t) - Derivative(vx(t), t)
 -omega_c*vx(t) - Derivative(vy(t), t)
                 -Derivative(vz(t), t)

In [209]:
simple_case = dsolve(simple_eq[1:2])
push!(simple_case, subs(dsolve(simple_eq[3]), C1=>C3))
map(simplify, simple_case)

3-element Array{SymPy.Sym,1}:
 Eq(vx(t), omega_c*(C1*sin(omega_c*t) + C2*cos(omega_c*t)))
 Eq(vy(t), omega_c*(C1*cos(omega_c*t) - C2*sin(omega_c*t)))
                                              Eq(C3, vz(t))

In [210]:
@syms vpara vperp
sub_eq = subs.(simple_case, C3=>vpara, C1=>0, C2=>vperp/omega_c)
cur_r = integrate.(map(rhs,sub_eq), t)

3-element Array{SymPy.Sym,1}:
 vperp*sin(omega_c*t)/omega_c
 vperp*cos(omega_c*t)/omega_c
                      t*vpara

In [211]:
using Plots
plotlyjs()

Plots.PlotlyJSBackend()

In [237]:
n_periods = 4
points_per_period = 40

cur_plot = map( cur_pos -> map(
    cur_t -> SymPy.N(subs(cur_pos, vpara => 1, vperp => 1, omega_c =>1, t=>cur_t)),
    linspace(0,n_periods*2*pi, n_periods*points_per_period)
), cur_r)

plot(cur_plot...)

In [214]:
plot()
for cur_period in 1:n_periods
    cur_range = 1:points_per_period
    cur_range += (cur_period-1) * points_per_period
    plot!(cur_plot[1][cur_range], cur_plot[2][cur_range])
end
plot!()

#### Uniform B w/ E ≠ 0

In [223]:
ez_eq = subs.(expand.(gyromotion()), bx=>0, by=>0, ey=>0, q*bz/m => omega_c)

3-element Array{SymPy.Sym,1}:
 ex*q/m + omega_c*vy(t) - Derivative(vx(t), t)
         -omega_c*vx(t) - Derivative(vy(t), t)
                 ez*q/m - Derivative(vz(t), t)

In [225]:
ez_case = dsolve(ez_eq[1:2])
push!(ez_case, subs(dsolve(ez_eq[3]), C1=>C3))
map(simplify, ez_case)

3-element Array{SymPy.Sym,1}:
                          Eq(vx(t), omega_c*(C1*sin(omega_c*t) + C2*cos(omega_c*t)))
 Eq(vy(t), C1*omega_c*cos(omega_c*t) - C2*omega_c*sin(omega_c*t) + ex*q/(m*omega_c))
                                                            Eq(vz(t), C3 + ez*q*t/m)

In [236]:
sub_eq_2 = subs.(ez_case, C3=>vpara, C1=>0, C2=>vperp/omega_c)
cur_r_2 = integrate.(map(rhs,sub_eq_2), t)

3-element Array{SymPy.Sym,1}:
                      vperp*sin(omega_c*t)/omega_c
 ex*q*t/(m*omega_c) + vperp*cos(omega_c*t)/omega_c
                          ez*q*t^2/(2*m) + t*vpara

#### ex ≠ 0 & ez ≠ 0

In [344]:
n_periods = 4
points_per_period = 40

cur_plot = map( cur_pos -> map(
    cur_t -> SymPy.N(subs(cur_pos, vpara => 1, vperp => 1, omega_c =>2, t=>cur_t, ex=>1/25, ez=>100, q=>1, m=>1)),
    linspace(0,n_periods*2*pi, n_periods*points_per_period)
), cur_r_2)

plot(cur_plot...)

In [345]:
plot()
for cur_period in 1:n_periods
    cur_range = 1:points_per_period
    cur_range += (cur_period-1) * points_per_period
    plot!(cur_plot[1][cur_range], cur_plot[2][cur_range])
end
plot!()

#### ez = 1 w/ ex = 0

In [333]:
n_periods = 4
points_per_period = 40

cur_plot_2 = map( cur_pos -> map(
    cur_t -> SymPy.N(subs(cur_pos, vpara => 1, vperp => 1, omega_c =>1.001, t=>cur_t, ex=>0, ez=>1, q=>1, m=>1)),
    linspace(0,n_periods*2*pi, n_periods*points_per_period)
), cur_r_2)

plot(cur_plot_2...)

In [334]:
plot()
for cur_period in 1:n_periods
    cur_range = 1:2:points_per_period
    cur_range += (cur_period-1) * points_per_period
    plot!(cur_plot_2[1][cur_range], cur_plot_2[2][cur_range], line=0, marker=(:d,n_periods-cur_period+1))
end
plot!()

#### ex = 1 w/ ez = 0

In [338]:
n_periods = 4
points_per_period = 40

cur_plot = map( cur_pos -> map(
    cur_t -> SymPy.N(subs(cur_pos, vpara => 1, vperp => 1, omega_c =>1, t=>cur_t, ex=>1/25, ez=>0, q=>1, m=>1)),
    linspace(0,n_periods*2*pi, n_periods*points_per_period)
), cur_r_2)

plot(cur_plot...)

In [339]:
plot()
for cur_period in 1:n_periods
    cur_range = 1:points_per_period
    cur_range += (cur_period-1) * points_per_period
    plot!(cur_plot[1][cur_range], cur_plot[2][cur_range])
end
plot!()